In [17]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import time 

import torch
import mrmr
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

In [18]:
# Constants
DATA_PATH = './data/'
DATASET_PATH = DATA_PATH + 'uci-data/'
MODELS_PATH = DATA_PATH + 'models/raw-models/'

In [19]:
complete_set = pd.read_csv(DATA_PATH + "self-calculated/raw-data/complete-filtered.txt", sep='\s+', header=None)
complete_set.columns = ['bodyAccX', 'gravityAccX', 'bodyAccY', 'gravityAccY', 'bodyAccZ', 'gravityAccZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]
train_set, test_set = train_test_split(complete_set, test_size=0.3)

print(train_set.shape)
print(test_set.shape)

(570929, 11)
(244685, 11)


In [20]:
# Creating model
device = None

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

class RawDataModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(10, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Linear(128, 32),
            nn.SiLU(),
            nn.Linear(32, 8),
            nn.Softmax()  # or nn.Softmax(dim=1) for multi-class classification
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
model = RawDataModel().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [21]:
# Training and testing function

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y = y.squeeze()
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            y = y.squeeze()
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
# Dataset preparation
class HAPTDataset(Dataset):
    def __init__(self, dataset, features, label):
        self.data = torch.tensor(dataset[features].values, dtype=torch.float32)
        self.labels = torch.tensor(dataset[label].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
train_dataset = HAPTDataset(train_set, train_set.columns[:-1], 'activity')
test_dataset = HAPTDataset(test_set, test_set.columns[:-1], 'activity')

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [23]:
epochs = 5
perf_timer = time.perf_counter()
perf_acc = ""

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    perf_acc = test(test_dataloader, model, loss_function)
    
print(perf_acc)
perf_timer = time.perf_counter() - perf_timer
print("Done!")

Epoch 1
-------------------------------
loss: 1.818553  [    8/570929]
loss: 1.769326  [  808/570929]
loss: 1.766707  [ 1608/570929]
loss: 1.979262  [ 2408/570929]
loss: 1.931402  [ 3208/570929]
loss: 2.074153  [ 4008/570929]
loss: 1.777011  [ 4808/570929]
loss: 2.128045  [ 5608/570929]
loss: 1.839279  [ 6408/570929]
loss: 1.663230  [ 7208/570929]
loss: 1.370289  [ 8008/570929]
loss: 1.961900  [ 8808/570929]
loss: 1.894720  [ 9608/570929]
loss: 1.756287  [10408/570929]
loss: 1.579127  [11208/570929]
loss: 1.960388  [12008/570929]
loss: 1.671329  [12808/570929]
loss: 1.728943  [13608/570929]
loss: 1.442723  [14408/570929]
loss: 1.338519  [15208/570929]
loss: 1.822840  [16008/570929]
loss: 1.496948  [16808/570929]
loss: 1.596454  [17608/570929]
loss: 1.891269  [18408/570929]
loss: 1.772888  [19208/570929]
loss: 1.957178  [20008/570929]
loss: 1.699071  [20808/570929]
loss: 1.769015  [21608/570929]
loss: 1.801247  [22408/570929]
loss: 1.713297  [23208/570929]
loss: 1.784326  [24008/570929]

KeyboardInterrupt: 